In [ ]:
import pyodbc
import pandas as pd

In [ ]:
#def SQLconnect():
conn = pyodbc.connect('Driver={SQL Server};'
                  'Server=MSI\SQLEXPRESS;'
                  'Database=STUDENTS;'
                  'Trusted_Connection=yes;')
conn

In [ ]:
#hiển thị danh sách sách
def info():
    sach=pd.DataFrame(columns=['Sách_id','Tên sách', 'Thể loại', 'Năm xuất bản', 'Số trang', 'Nhà xuất bản', 'Giá tiền'])
    read_sql = """
    select Sach_id,Ten_sach,Nam_xuat_ban,So_trang,Gia_tien,Ten_the_loai, Ten_nha_xuat_ban
    from Sach left join The_loai on The_loai = The_loai_id left join Nha_xuat_ban
    on Nha_xuat_ban = Nha_xuat_ban_id
    """
    #Get Cursor 
    cursor = conn.cursor()
    cursor.execute(read_sql)
    result = cursor.fetchall()
    for i in range(len(result)):
        row={}
        row['Sách_id'] = result[i][0]
        row['Tên sách'] = result[i][1]
        row['Thể loại'] = result[i][5]
        row['Năm xuất bản'] = result[i][2]
        row['Số trang'] = result[i][3]
        row['Nhà xuất bản'] = result[i][6]
        row['Giá tiền'] = result[i][4]
        sach=sach.append(row,ignore_index=True)
    print(sach)

In [ ]:
#thêm sách
def insert_book():
    ten = input('Nhập tên sách')

    cursor = conn.cursor()
    cursor.execute('SELECT * FROM The_loai')
    list_type = cursor.fetchall()
    the_loai_id = []
    ten_the_loai = []
    for i in range (len(list_type)):
        the_loai_id.append(list_type[i][0]) 
        ten_the_loai.append(list_type[i][1]) 
    the_loai_id = pd.Series(the_loai_id)
    ten_the_loai =pd.Series(ten_the_loai)
    Type = pd.DataFrame ({'The_loai_id':the_loai_id,'Ten_the_loai':ten_the_loai})
    print(Type)
    the_loai = input('Nhập mã thể loại')

    nam_xuat_ban = int(input('Nhập năm xuất bản'))
    so_trang = input('Nhập số trang sách')

    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Nha_xuat_ban')
    list_printer = cursor.fetchall()
    nha_xuat_ban_id = []
    ten_nha_xuat_ban = []
    for i in range (len(list_printer)):
        nha_xuat_ban_id.append(list_printer[i][0]) 
        ten_nha_xuat_ban.append(list_printer[i][1]) 
    nha_xuat_ban_id = pd.Series(nha_xuat_ban_id)
    ten_nha_xuat_ban =pd.Series(ten_nha_xuat_ban)
    Printer = pd.DataFrame ({'The_loai_id':nha_xuat_ban_id,'Ten_the_loai':ten_nha_xuat_ban})
    print(Printer)
    nha_xuat_ban = input('Nhập mã nhà xuất bản')

    gia_tien = int(input('Nhập giá tiền'))
    insertion_sql_with_params = """
    Insert into Sach (Ten_sach,The_loai,Nam_xuat_ban,So_trang,Nha_xuat_ban,Gia_tien)
    values(?,?,?,?,?,?)
    """
    #Get Cursor 
    cursor = conn.cursor()
    cursor.execute(insertion_sql_with_params,ten,the_loai,nam_xuat_ban,so_trang,nha_xuat_ban, gia_tien)
    #Phải commit
    conn.commit()
    print('Thêm thông tin sách thành công')

In [ ]:
#cập nhật sách
def update_book():
    sach_id=int(input('Nhập id của sách, bạn muốn sửa thông tin: '))
    validation_sql ="select * from Sach where Sach_id =?"
    cursor = conn.cursor()
    cursor.execute(validation_sql, sach_id)
    result = cursor.fetchone()
    if result is None:
        print(f'Id của sách ={sach_id} không tồn tại. Nhập lại để cập nhật')
    else:
        print('Tên sách ',result[1])
        print('Thể loại ',result[2])

        cursor = conn.cursor()
        cursor.execute('SELECT * FROM The_loai')
        list_type = cursor.fetchall()
        the_loai_id = []
        ten_the_loai = []
        for i in range (len(list_type)):
            the_loai_id.append(list_type[i][0]) 
            ten_the_loai.append(list_type[i][1]) 
        the_loai_id = pd.Series(the_loai_id)
        ten_the_loai =pd.Series(ten_the_loai)
        Type = pd.DataFrame ({'The_loai_id':the_loai_id,'Ten_the_loai':ten_the_loai})
        print(Type)

        print('Năm xuất bản ',result[3])
        print('Số trang ',result[4])
        print('Nhà xuất bản ',result[5])

        cursor = conn.cursor()
        cursor.execute('SELECT * FROM Nha_xuat_ban')
        list_printer = cursor.fetchall()
        nha_xuat_ban_id = []
        ten_nha_xuat_ban = []
        for i in range (len(list_printer)):
            nha_xuat_ban_id.append(list_printer[i][0]) 
            ten_nha_xuat_ban.append(list_printer[i][1]) 
        nha_xuat_ban_id = pd.Series(nha_xuat_ban_id)
        ten_nha_xuat_ban =pd.Series(ten_nha_xuat_ban)
        Printer = pd.DataFrame ({'Nha_xuat_ban_id':nha_xuat_ban_id,'Ten_nha_xuat_ban':ten_nha_xuat_ban})
        print(Printer)

        print('Giá tiền ',result[6])
        ten = input('Nhập tên sách (Để trống nếu không cập nhật trường này):')
        the_loai = input('Nhập mã thể loại (Để trống nếu không cập nhật trường này):')
        nam_xuat_ban = input('Nhập năm xuất bản (Để trống nếu không cập nhật trường này):')
        so_trang = input('Nhập số trang (Để trống nếu không cập nhật trường này):')
        nha_xuat_ban = input('Nhập mã nhà xuất bản (Để trống nếu không cập nhật trường này):')
        gia_tien = input('Nhập gia tien (Để trống nếu không cập nhật trường này):')
        if ten == '':
            ten = result[1]
        if the_loai =='':
            the_loai = result[2]
        else:
            the_loai = int(the_loai)
        if nam_xuat_ban =='':
            nam_xuat_ban = result[3]
        else:
            nam_xuat_ban = int(nam_xuat_ban)
        if so_trang == '':
            so_trang= result[4]
        else:
            so_trang = int(so_trang)
        if nha_xuat_ban =='':
            nha_xuat_ban = result[5]
        else:
            nha_xuat_ban = int(nha_xuat_ban)
        if gia_tien =='':
            gia_tien = result[6]
        else:
            gia_tien = int(gia_tien)
        update_sql ="""Update Sach set Ten_sach = ?, 
        The_loai =?,Nam_xuat_ban = ?, So_trang = ?, Nha_xuat_ban =?,
        Gia_tien =? where Sach_id = ?
        """
        #Get Cursor 
        cursor = conn.cursor()
        cursor.execute(update_sql, ten,the_loai,nam_xuat_ban,so_trang,nha_xuat_ban, gia_tien,sach_id)
        #Phải commit
        conn.commit()
        print('Cập nhật thông tin thành công')

In [ ]:
#xóa sách
def del_book():
    sach_id=int(input('Nhập id của sách, bạn muốn xóa thông tin: '))
    validation_sql ="select * from Sach where Sach_id =?"
    cursor = conn.cursor()
    cursor.execute(validation_sql, sach_id)
    result = cursor.fetchone()
    if result is None:
        print(f'Id của sách ={sach_id} không tồn tại. Nhập lại để xóa')
    else:
        confirm = input(f'Bạn chắc chắn muốn xóa thông tin của sách {result[1]} (y/n)?')
        if confirm.lower()=='y':
            delete_sql = 'Delete from Sach where Sach_id = ?'
            cursor = conn.cursor()
            cursor.execute(delete_sql, sach_id)
            #Phải commit
            conn.commit()
            print(f'xóa thông tin của sách {result[1]} thành công')

In [ ]:
#tìm kiếm sách
def search_book():
    print('=== 1. Tìm theo tên                     ===')
    print('=== 2. Tìm theo thể loại                ===')
    print('=== 3. Tìm theo năm xuất bản            ===')
    print('=== 4. Tìm theo số trang                ===')
    print('=== 5. Tìm theo nhà xuất bản            ===')
    print('=== 6. Tìm theo giá tiền                ===')
    choice = input('Nhập lựa chọn: ')
    if choice =='1':
        key_search = input('Nhập từ khóa ')
        search_sql = f"""
    select Ten_sach,Nam_xuat_ban,So_trang,Gia_tien,Ten_the_loai, Ten_nha_xuat_ban
    from Sach left join The_loai on The_loai = The_loai_id left join Nha_xuat_ban
    on Nha_xuat_ban = Nha_xuat_ban_id where Ten_sach like '%{key_search}%'
    """
    #Get Cursor 
        cursor = conn.cursor()
        cursor.execute(search_sql)
        result = cursor.fetchall()
        conn.commit()
        print(result)
    elif choice =='2':
        key_search = input('Nhập từ khóa: ')
        search_sql = f"""
        select The_loai_id from The_loai where Ten_the_loai like N'%{key_search}%'
        """
        cursor = conn.cursor()
        cursor.execute(search_sql)
        result = cursor.fetchall()
        conn.commit()
        for i in range(len(result)):
            search_sql_full = """
            select Ten_sach,Nam_xuat_ban,So_trang,Gia_tien,Ten_the_loai, Ten_nha_xuat_ban
            from Sach left join The_loai on The_loai = The_loai_id left join Nha_xuat_ban
            on Nha_xuat_ban = Nha_xuat_ban_id where The_loai = ?
            """
        #Get Cursor 
            cursor = conn.cursor()
            cursor.execute(search_sql_full,result[i][0])
            full_result = cursor.fetchall()
            conn.commit()
            print(full_result)
    elif choice =='3':
        key_search = input('Nhập từ khóa ')
        search_sql = f"""
    select Ten_sach,Nam_xuat_ban,So_trang,Gia_tien,Ten_the_loai, Ten_nha_xuat_ban
    from Sach left join The_loai on The_loai = The_loai_id left join Nha_xuat_ban
    on Nha_xuat_ban = Nha_xuat_ban_id where Nam_xuat_ban like '%{key_search}%'
    """
        #Get Cursor 
        cursor = conn.cursor()
        cursor.execute(search_sql)
        result = cursor.fetchall()
        conn.commit()
        print(result)
    elif choice =='4':
        key_search = input('Nhập từ khóa ')
        search_sql = f"""
    select Ten_sach,Nam_xuat_ban,So_trang,Gia_tien,Ten_the_loai, Ten_nha_xuat_ban
    from Sach left join The_loai on The_loai = The_loai_id left join Nha_xuat_ban
    on Nha_xuat_ban = Nha_xuat_ban_id where So_trang like '%{key_search}%'
    """
        #Get Cursor 
        cursor = conn.cursor()
        cursor.execute(search_sql)
        result = cursor.fetchall()
        conn.commit()
        print(result)
    elif choice =='5':
        key_search = input('Nhập từ khóa: ')
        search_sql = f"""
        select Nha_xuat_ban_id from Nha_xuat_ban where Ten_nha_xuat_ban like N'%{key_search}%'
        """
        cursor = conn.cursor()
        cursor.execute(search_sql)
        result = cursor.fetchall()
        conn.commit()
        for i in range(len(result)):
            search_sql_full = """
            select Ten_sach,Nam_xuat_ban,So_trang,Gia_tien,Ten_the_loai, Ten_nha_xuat_ban
            from Sach left join The_loai on The_loai = The_loai_id left join Nha_xuat_ban
            on Nha_xuat_ban = Nha_xuat_ban_id where Nha_xuat_ban = ?
            """
        #Get Cursor 
            cursor = conn.cursor()
            cursor.execute(search_sql_full,result[i][0])
            full_result = cursor.fetchall()
            conn.commit()
            print(full_result)
    elif choice =='6':
        key_search = input('Nhập từ khóa ')
        search_sql = f"""
    select Ten_sach,Nam_xuat_ban,So_trang,Gia_tien,Ten_the_loai, Ten_nha_xuat_ban
    from Sach left join The_loai on The_loai = The_loai_id left join Nha_xuat_ban
    on Nha_xuat_ban = Nha_xuat_ban_id where Gia_tien like '%{key_search}%'
    """
        #Get Cursor 
        cursor = conn.cursor()
        cursor.execute(search_sql)
        result = cursor.fetchall()
        conn.commit()
        print(result)
    else:
        print('Lựa chọn sai!')

In [ ]:
#hiển thị danh sách thể loại
def info_type():
    the_loai=pd.DataFrame(columns=['Id thể loại','Tên thể loại'])
    read_type = """select * from The_loai"""
    #Get Cursor 
    cursor = conn.cursor()
    cursor.execute(read_type)
    result = cursor.fetchall()
    for i in range(len(result)):
        row={}
        row['Id thể loại'] = result[i][0]
        row['Tên thể loại'] = result[i][1]
        the_loai=the_loai.append(row,ignore_index=True)
    print(the_loai)

In [ ]:
#hiển thị danh sách nhà xuất bản
def info_printer():
    nha_xuat_ban=pd.DataFrame(columns=['Id nhà xuất bản','Tên nhà xuất bản'])
    read_printer = """select * from Nha_xuat_ban"""
    #Get Cursor 
    cursor = conn.cursor()
    cursor.execute(read_printer)
    result = cursor.fetchall()
    for i in range(len(result)):
        row={}
        row['Id nhà xuất bản'] = result[i][0]
        row['Tên nhà xuất bản'] = result[i][1]
        nha_xuat_ban=nha_xuat_ban.append(row,ignore_index=True)
    print(nha_xuat_ban)

In [ ]:
#thêm thể loại
def insert_type():
    the_loai = input('Nhập tên thể loại ')
    insertion_sql_type = """
    Insert into The_loai (Ten_the_loai)
    values (?)
    """
    #Get Cursor 
    cursor = conn.cursor()
    cursor.execute(insertion_sql_type,the_loai)
    #Phải commit
    conn.commit()
    print('Thêm thông tin thể loại thành công')

In [ ]:
#thêm nhà xuất bản
def insert_printer():
    nha_xuat_ban = input('Nhập tên nhà xuất bản ')
    insertion_sql_type = """
    Insert into Nha_xuat_ban (Ten_nha_xuat_ban)
    values (?)
    """
    #Get Cursor 
    cursor = conn.cursor()
    cursor.execute(insertion_sql_type,nha_xuat_ban)
    #Phải commit
    conn.commit()
    print('Thêm thông tin nhà xuất bản thành công')

In [ ]:
#cập nhật thể loại
def update_type():
    type_id=int(input('Nhập id của thể loại, bạn muốn sửa thông tin: '))
    validation_sql ="select * from The_loai where The_loai_id =?"
    cursor = conn.cursor()
    cursor.execute(validation_sql, type_id)
    result = cursor.fetchone()
    if result is None:
        print(f'Id của thể loại ={type_id} không tồn tại. Nhập lại để cập nhật')
    else:
        print('Thể loại ',result[1])
        the_loai = input('Nhập thể loại mới ')
        update_sql_type="""Update The_loai set Ten_the_loai = ? where The_loai_id = ?
        """
        #Get Cursor 
        cursor = conn.cursor()
        cursor.execute(update_sql_type, the_loai,type_id)
        #Phải commit
        conn.commit()
        print('Cập nhật thông tin thành công')

In [ ]:
#cập nhật nhà xuất bản
def update_printer():
    printer_id=int(input('Nhập id của nhà xuất bản, bạn muốn sửa thông tin: '))
    validation_sql ="select * from Nha_xuat_ban where Nha_xuat_ban_id =?"
    cursor = conn.cursor()
    cursor.execute(validation_sql, printer_id)
    result = cursor.fetchone()
    if result is None:
        print(f'Id của nhà xuất bản ={printer_id} không tồn tại. Nhập lại để cập nhật')
    else:
        print('Nhà xuất bản ',result[1])
        nha_xuat_ban = input('Nhập nhà xuất bản mới ')
        update_sql_type="""Update Nha_xuat_ban set Ten_nha_xuat_ban = ? where Nha_xuat_ban_id = ?
        """
        #Get Cursor 
        cursor = conn.cursor()
        cursor.execute(update_sql_type, nha_xuat_ban,printer_id)
        #Phải commit
        conn.commit()
        print('Cập nhật thông tin thành công')

In [ ]:
#Xóa thể loại
def del_type():
    type_id=int(input('Nhập id của thể loại, bạn muốn xóa thông tin: '))
    validation_sql ="select * from The_loai where The_loai_id =?"
    cursor = conn.cursor()
    cursor.execute(validation_sql, type_id)
    result = cursor.fetchone()
    if result is None:
        print(f'Id của thể loại ={type_id} không tồn tại. Nhập lại để xóa')
    else:
        confirm = input(f'Bạn chắc chắn muốn xóa thông tin của thể loại {result[1]} (y/n)?')
        if confirm.lower()=='y':
            delete_sql_type = 'Delete from The_loai where The_loai_id =?'
            cursor = conn.cursor()
            cursor.execute(delete_sql_type, type_id)
            #Phải commit
            conn.commit()
            print(f'xóa thông tin của thể loại {result[1]} thành công')

In [ ]:
#Xóa nhà xuất bản
def del_printer():
    printer_id=int(input('Nhập id của nhà xuất bản, bạn muốn xóa thông tin: '))
    validation_sql ="select * from Nha_xuat_ban where Nha_xuat_ban_id =?"
    cursor = conn.cursor()
    cursor.execute(validation_sql, printer_id)
    result = cursor.fetchone()
    if result is None:
        print(f'Id của nhà xuất bản ={printer_id} không tồn tại. Nhập lại để xóa')
    else:
        confirm = input(f'Bạn chắc chắn muốn xóa thông tin của nhà xuất bản {result[1]} (y/n)?')
        if confirm.lower()=='y':
            delete_sql_printer = 'Delete from Nha_xuat_ban where Nha_xuat_ban_id =?'
            cursor = conn.cursor()
            cursor.execute(delete_sql_printer, printer_id)
            #Phải commit
            conn.commit()
            print(f'xóa thông tin của nhà xuất bản {result[1]} thành công')

In [ ]:
class Program:
    def run(self):
        while 1:
            print('================ QUẢN LÝ SÁCH ===============')
            print('=== 1. Hiển thị danh sách sách            ===')
            print('=== 2. Thêm sách                          ===')
            print('=== 3. Chỉnh sửa thông tin sách           ===')
            print('=== 4. Xoá thông tin sách                 ===')
            print('=== 5. Tìm kiếm thông tin                 ===')
            print('=== 6. Hiển thị danh mục thể loại         ===')
            print('=== 7. Thêm thể loại                      ===')
            print('=== 8. Chỉnh sửa thông tin thể loại       ===')
            print('=== 9. Xóa thông tin thể loại             ===')
            print('=== 10. Hiển thị danh mục nhà xuất bản    ===')
            print('=== 11. Thêm nhà xuất bản                 ===')
            print('=== 12. Chỉnh sửa thông tin nhà xuất bản  ===')
            print('=== 13. Xóa thông tin nhà xuất bản        ===')
            print('=== 14. Đóng chương trình                 ===')
            print('=============================================')
            choice = input('Nhập lựa chọn: ')
            if choice=='1':
                info()
            elif choice =='2':
                insert_book()
            elif choice =='3':
                update_book()
            elif choice =='4':
                del_book()
            elif choice =='5':
                search_book()
            elif choice =='6':
                info_type()
            elif choice =='7':
                insert_type()
            elif choice =='8':
                update_type()
            elif choice =='9':
                del_type()
            elif choice =='10':
                info_printer()
            elif choice =='11':
                insert_printer()
            elif choice =='12':
                update_printer()
            elif choice =='13':
                del_printer()
            elif choice=='14':
                confirm = input('Bạn muốn đóng chương trình (y/n)? ')
                if confirm=='y' or confirm=='Y':
                    break
            else:
                print('Lựa chon sai! Bấm 1 ->7 để lựa chọn các tác vụ')
            input('Nhấn enter để tiếp tục ...')
        print('Chương trình kết thúc...')
            

In [ ]:
main = Program()
main.run()